In [2]:
# import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# read data
df = pd.read_csv("ML_data_unscaled_dummys_encoded.csv")
df = df.drop('Entry_ID',axis=1)
df.head()

,Age,Hours_per_day,BPM,Anxiety,Depression,Insomnia,OCD,While_working_No,While_working_Yes,Instrumentalist_No,...,Rock_Rarely,Rock_Sometimes,Rock_Very frequently,Video game music_Never,Video game music_Rarely,Video game music_Sometimes,Video game music_Very frequently,Music_effects_Improve,Music_effects_No effect,Music_effects_Worsen
0,18.0,4.0,132.0,7.0,7.0,10.0,2.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,61.0,2.5,84.0,9.0,7.0,3.0,3.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,18.0,4.0,107.0,7.0,2.0,5.0,9.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,18.0,5.0,86.0,8.0,8.0,7.0,7.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,18.0,3.0,66.0,4.0,8.0,6.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511 entries, 0 to 510
Data columns (total 94 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Age                               511 non-null    float64
 1   Hours_per_day                     511 non-null    float64
 2   BPM                               511 non-null    float64
 3   Anxiety                           511 non-null    float64
 4   Depression                        511 non-null    float64
 5   Insomnia                          511 non-null    float64
 6   OCD                               511 non-null    float64
 7   While_working_No                  511 non-null    float64
 8   While_working_Yes                 511 non-null    float64
 9   Instrumentalist_No                511 non-null    float64
 10  Instrumentalist_Yes               511 non-null    float64
 11  Composer_No                       511 non-null    float64
 12  Composer

## Mental improvement model

In [18]:
# collapse target columns
target_cols = ['Music_effects_Improve','Music_effects_No effect','Music_effects_Worsen']
target_col = df[target_cols].idxmax(1)
target_col.value_counts()

Music_effects_Improve      385
Music_effects_No effect    113
Music_effects_Worsen        13
dtype: int64

In [19]:
# encode target
y = target_col.replace({'Music_effects_Improve':1,'Music_effects_No effect':0,'Music_effects_Worsen':0}).values

In [23]:
# split data into features and target
X = df.drop(columns=target_cols).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    # activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32, max_value=512, step=32), activation=hp.Choice('activation',['relu','tanh','sigmoid'])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation',['relu','tanh','sigmoid'])))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    directory='imp_hyp',
    overwrite=False,
    project_name='imp_hyp')

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test))

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
best_model.save("trained_dep.h5")